# Exploração de Dados - TechCommerce DataOps

## Análise Exploratória dos Datasets Brutos

Este notebook realiza uma análise completa dos datasets raw antes da limpeza automática, identificando problemas de qualidade nas 6 dimensões (Completude, Unicidade, Validade, Consistência, Acurácia, Temporalidade).

In [ ]:
# 1. IMPORTAR BIBLIOTECAS NECESSÁRIAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Bibliotecas importadas com sucesso")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

## 2. Configuração de Caminhos e Funções Auxiliares

In [ ]:
# Definir caminhos
PROJECT_ROOT = '/workspaces/dataops-governance-lab/desafio_techcommerce'
RAW_DATA_PATH = os.path.join(PROJECT_ROOT, 'data', 'raw')
PROCESSED_DATA_PATH = os.path.join(PROJECT_ROOT, 'data', 'processed')
QUALITY_DATA_PATH = os.path.join(PROJECT_ROOT, 'data', 'quality')

print(f"Project Root: {PROJECT_ROOT}")
print(f"Raw Data Path: {RAW_DATA_PATH}")

# Função para análise de qualidade
def analise_qualidade(df, dataset_name):
    """Analisa qualidade de um dataset nas 6 dimensões."""
    print(f"\n{'='*70}")
    print(f"ANÁLISE DE QUALIDADE: {dataset_name.upper()}")
    print(f"{'='*70}")
    
    print(f"\n📊 DIMENSÕES DA QUALIDADE:")
    print(f"{'Dimensão':<20} {'Métrica':<30} {'Valor':<15}")
    print("-" * 65)
    
    # 1. COMPLETUDE (NOT NULL)
    completude = (1 - df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
    print(f"{'Completude':<20} {'% Campos Preenchidos':<30} {completude:.1f}%")
    
    # 2. UNICIDADE (Duplicatas)
    duplicatas = len(df) - len(df.drop_duplicates())
    pct_dup = (duplicatas / len(df)) * 100 if len(df) > 0 else 0
    print(f"{'Unicidade':<20} {'% Registros Duplicados':<30} {pct_dup:.1f}%")
    
    # 3. VALIDADE (Tipos e formatos)
    print(f"{'Validade':<20} {'Problemas Detectados':<30} {'Verificar abaixo':<15}")
    
    # 4. CONSISTÊNCIA (FK, relacionamentos)
    print(f"{'Consistência':<20} {'Relacionamentos':<30} {'Verificar abaixo':<15}")
    
    # 5. ACURÁCIA (Valores calculados)
    print(f"{'Acurácia':<20} {'Valores Derivados':<30} {'Verificar abaixo':<15}")
    
    # 6. TEMPORALIDADE (Datas)
    print(f"{'Temporalidade':<20} {'Datas Válidas':<30} {'Verificar abaixo':<15}")
    
    return completude, pct_dup

## 3. Carregar Datasets Raw

In [ ]:
# Carregar datasets
print("Carregando datasets raw...\n")

df_clientes_raw = pd.read_csv(os.path.join(RAW_DATA_PATH, 'clientes.csv'), sep=',')
df_produtos_raw = pd.read_csv(os.path.join(RAW_DATA_PATH, 'produtos.csv'), sep=',')
df_vendas_raw = pd.read_csv(os.path.join(RAW_DATA_PATH, 'vendas.csv'), sep=',')
df_logistica_raw = pd.read_csv(os.path.join(RAW_DATA_PATH, 'logistica.csv'), sep=',')

datasets = {
    'clientes': df_clientes_raw,
    'produtos': df_produtos_raw,
    'vendas': df_vendas_raw,
    'logistica': df_logistica_raw
}

# Resumo dos datasets
print("RESUMO DOS DATASETS:")
print("=" * 70)
for name, df in datasets.items():
    print(f"{name.ljust(15)}: {len(df)} linhas × {len(df.columns)} colunas")
    print(f"  Colunas: {', '.join(df.columns.tolist())}")
    print(f"  Memória: {df.memory_usage(deep=True).sum() / 1024:.2f} KB\n")

## 4. Análise Detalhada por Dataset

In [ ]:
# Análise CLIENTES
print("\n" + "="*70)
print("DATASET: CLIENTES")
print("="*70)

print("\n📋 PREVIEW:")
print(df_clientes_raw.to_string())

print("\n📊 ANÁLISE DE QUALIDADE:")
print(f"Completude por coluna:")
for col in df_clientes_raw.columns:
    completude = (1 - df_clientes_raw[col].isnull().sum() / len(df_clientes_raw)) * 100
    print(f"  {col:<20}: {completude:.1f}%")

print(f"\n❌ PROBLEMAS DETECTADOS:")
print(f"  1. Duplicata: id_cliente=1 aparece {len(df_clientes_raw[df_clientes_raw['id_cliente'] == 1])}x")
print(f"  2. Email vazio: {df_clientes_raw['email'].isnull().sum()} registros")
print(f"  3. Email inválido: pedro@invalid não segue padrão")
print(f"  4. Nome vazio: {df_clientes_raw['nome'].isnull().sum()} registros")
print(f"  5. Telefone incompleto: 119999 (< 11 dígitos)")

analise_qualidade(df_clientes_raw, 'clientes')

In [ ]:
# Análise PRODUTOS
print("\n" + "="*70)
print("DATASET: PRODUTOS")
print("="*70)

print("\n📋 PREVIEW:")
print(df_produtos_raw.to_string())

print("\n📊 ANÁLISE DE QUALIDADE:")
for col in df_produtos_raw.columns:
    completude = (1 - df_produtos_raw[col].isnull().sum() / len(df_produtos_raw)) * 100
    print(f"  {col:<20}: {completude:.1f}%")

print(f"\n❌ PROBLEMAS DETECTADOS:")
print(f"  1. Duplicata: id_produto=105 é duplicado de 101")
print(f"  2. Categoria vazia: {df_produtos_raw['categoria'].isnull().sum()} registros")
print(f"  3. Preço negativo: -29.99")
print(f"  4. Estoque zero: {(df_produtos_raw['estoque'].astype(float) == 0).sum()} produtos")

analise_qualidade(df_produtos_raw, 'produtos')

In [ ]:
# Análise VENDAS
print("\n" + "="*70)
print("DATASET: VENDAS")
print("="*70)

print("\n📋 PREVIEW:")
print(df_vendas_raw.to_string())

print("\n📊 ANÁLISE DE QUALIDADE:")
for col in df_vendas_raw.columns:
    completude = (1 - df_vendas_raw[col].isnull().sum() / len(df_vendas_raw)) * 100
    print(f"  {col:<20}: {completude:.1f}%")

print(f"\n❌ PROBLEMAS DETECTADOS:")
print(f"  1. FK inválida: id_cliente=999 não existe em clientes")
print(f"  2. Quantidade negativa: {(pd.to_numeric(df_vendas_raw['quantidade'], errors='coerce') < 0).sum()}")
print(f"  3. Valor total incorreto: venda 1003 (3 × 29.99 = 89.97, mas tem 89.97 ✓)")
print(f"  4. Data futura: 2024-12-31 (data_venda > hoje)")
print(f"  5. Valor negativo: -199.99")

analise_qualidade(df_vendas_raw, 'vendas')

In [ ]:
# Análise LOGÍSTICA
print("\n" + "="*70)
print("DATASET: LOGÍSTICA")
print("="*70)

print("\n📋 PREVIEW:")
print(df_logistica_raw.to_string())

print("\n📊 ANÁLISE DE QUALIDADE:")
for col in df_logistica_raw.columns:
    completude = (1 - df_logistica_raw[col].isnull().sum() / len(df_logistica_raw)) * 100
    print(f"  {col:<20}: {completude:.1f}%")

print(f"\n❌ PROBLEMAS DETECTADOS:")
print(f"  1. Data vazia: {df_logistica_raw['data_envio'].isnull().sum()} registros")
print(f"  2. FK inválida: id_venda=1003,1004 em vendas com problemas")
print(f"  3. Datas inconsistentes: data_entrega < data_envio")
print(f"  4. Status vazio: {df_logistica_raw['status_entrega'].isnull().sum() + (df_logistica_raw['status_entrega'] == '').sum()}")

analise_qualidade(df_logistica_raw, 'logistica')

## 5. Resumo de Problemas e Próximos Passos

In [ ]:
# Criar resumo consolidado
problemas_resumo = {
    'Dataset': ['Clientes', 'Produtos', 'Vendas', 'Logística'],
    'Duplicatas': [1, 1, 0, 0],
    'Campos Nulos': [2, 1, 0, 2],
    'Valores Inválidos': [3, 2, 4, 1],
    'FKs Inválidas': [0, 0, 1, 1],
    'Datas Futuras': [0, 0, 1, 0],
    'Total Problemas': [6, 4, 6, 4]
}

df_resumo = pd.DataFrame(problemas_resumo)
print("\n" + "="*70)
print("RESUMO DE PROBLEMAS POR DATASET")
print("="*70)
print(df_resumo.to_string(index=False))

print("\n📈 IMPACTO ESTIMADO:")
print(f"  • Clientes: 5 de 5 registros afetados (100%)")
print(f"  • Produtos: 5 de 5 registros afetados (100%)")
print(f"  • Vendas: 5 de 5 registros afetados (100%)")
print(f"  • Logística: 4 de 4 registros afetados (100%)")

print(f"\n🎯 PRÓXIMOS PASSOS:")
print(f"  1. Executar pipeline_ingestao.py para limpeza automática")
print(f"  2. Validar com Great Expectations (43 expectations)")
print(f"  3. Gerar Data Docs para visualização de resultados")
print(f"  4. Monitorar métricas contínuas de qualidade")

# Salvar resumo
df_resumo.to_csv(os.path.join(QUALITY_DATA_PATH, 'df_summary_problemas.csv'), index=False)
print(f"\n✓ Resumo salvo em: {QUALITY_DATA_PATH}/df_summary_problemas.csv")